In [125]:
import pandas as pd
import multiprocessing as mp
import nltk
import pandas as pd
import numpy as np
import tensorflow as tf
import parmap
import swifter
from nltk.corpus import stopwords

In [126]:
topics_index_to_name_map = {
    0: 'Agriculture, animals, food and rural affairs',
    1: 'Asylum, immigration and nationality',
    2: 'Business, industry and consumers',
    3: 'Communities and families',
    4: 'Crime, civil law, justice and rights',
    5: 'Culture, media and sport',
    6: 'Defence',
    7: 'Economy and finance',
    8: 'Education',
    9: 'Employment and training',
    10: 'Energy and environment',
    11: 'European Union',
    12: 'Health services and medicine',
    13: 'Housing and planning',
    14: 'International affairs',
    15: 'Parliament, government and politics',
    16: 'Science and technology',
    17: 'Social security and pensions',
    18: 'Social services',
    19: 'Transport',
    20: 'Others'
}
topics_name_to_index_map = {y:x for x,y in topics_index_to_name_map.items()}

def strip_short2(text):
    return strip_short(text, minsize=4)


def preprocess_text(text):
    FILTERS = [lambda x: x.lower(), strip_multiple_whitespaces, strip_tags, strip_punctuation,
                   strip_non_alphanum, strip_numeric, strip_short2]
    return preprocess_string(text, FILTERS)

def preprocess(topic):
    ret = []
    topic = topic.strip()
    
    if '|' in topic:
        topics = topic.split('|')
        t = topics[0]
        t = t.strip()
        return topics_name_to_index_map[t]
        
    return topics_name_to_index_map[topic]

In [127]:
bbc_id = 54
year = 2016
excluding = '_no_Others'
years = [2016]
df_list = []

In [128]:
for year in years:
    print(year)
    df_list.append(pd.read_csv('../data/partitions/bert_partitions_{}_{}.csv'.format(bbc_id, year)))
    
transcripts = pd.concat(df_list)
transcripts = transcripts.drop(['Unnamed: 0'], axis=1)

2016


In [129]:
transcripts = transcripts.loc[(transcripts.Date.str.contains('may')) | (transcripts.Date.str.contains('apr'))]

In [130]:
partitions = []
for index, row in transcripts.iterrows():
    partition_string = row['partitioned_transcript']
    partition_date = row['Date']
    all_partitions = partition_string.split('\n---------------------\n')
    for partition in all_partitions:
        partitions.append((index, partition_date, partition))
partition_df = pd.DataFrame(partitions, columns=['partition_id', 'date', 'transcript'])

In [131]:
partition_df

,partition_id,date,transcript
0,521,2-apr-2016,between them created an artificially intellige...
1,521,2-apr-2016,The Xbox connects we are using in this space w...
2,521,2-apr-2016,Our home state of Arizona is overrun by home s...
3,521,2-apr-2016,The goal was to blur the lines between what’s ...
4,521,2-apr-2016,Don’t forget you can still immerse Don’t forge...
...,...,...,...
11860,1284,31-may-2016,What to know what a Donald Trump residency wou...
11861,1284,31-may-2016,We will be talking to one of its foreign polic...
11862,1284,31-may-2016,Tonight we reveal startling new figures showin...
11863,1284,31-may-2016,"Along with more affordable homes, Sadiq Khan h..."


In [132]:
stopW = stopwords.words('english')

def contains_european(x):
    y = x.lower()
    words = y.split()
    flag = 'e.u' in words or 'eu' in words or 'europeanunion' in words
    words = [word for word in words if word not in stopW]
    bigrams = list(nltk.bigrams(words))
    bigrams = [bigram[0] + '.' + bigram[1] for bigram in bigrams]
    
    flag = flag or 'european.union' in bigrams or 'europe.union' in bigrams
    
    return 'EU' if flag else 'non_EU'

In [133]:
partition_df['EU'] = partition_df['transcript'].apply(lambda x: contains_european(x))

In [134]:
partition_df = partition_df.loc[partition_df.EU == 'EU'].reset_index(drop=True)
partition_df

,partition_id,date,transcript,EU
0,521,2-apr-2016,Well... You are not reconciled and you now run...,EU
1,521,2-apr-2016,In my six months in the finance ministry I put...,EU
2,521,2-apr-2016,"But overshadowing much of the summit, the pros...",EU
3,521,2-apr-2016,A holding centre for migrants on the holding c...,EU
4,521,2-apr-2016,Greece is pushing ahead with plans to start de...,EU
...,...,...,...,...
567,1284,31-may-2016,Also tonight: This city wouldn’t be what it is...,EU
568,1284,31-may-2016,Leaving the European Union would allow Britain...,EU
569,1284,31-may-2016,"And there he was, telling his own side just wh...",EU
570,1284,31-may-2016,With just over three weeks to the EU referendu...,EU


In [135]:
brexit_file = pd.read_csv('./brexit_full_updated_0_1_only.csv')

politicians = []

count_zero = 0

politicians_party ={}

for index, row in brexit_file.iterrows():
    full_name = row['full name']
    party = row['party']
    
    last_name = full_name.split(" ")[-1]
    first_name = full_name.split(" ")[0]
    short_name = last_name.lower() + '.' + first_name.lower()

    if party == 0:
        count_zero += 1
    if party in [0, 1]:
        politicians.append(short_name)
        politicians_party[short_name] = party
        
print(len(politicians))
politicians = list(set(politicians))
print(len(politicians))

323
323


In [136]:
politicians

['borwick.victoria',
 'mcloughlin.patrick',
 'warburton.david',
 'jayawardena.ranil',
 'poulter.dan',
 'double.steve',
 'tyrie.andrew',
 'gillan.cheryl',
 'burt.alistair',
 'swayne.desmond',
 'smith.julian',
 'mcpartland.stephen',
 'argar.edward',
 'gauke.david',
 'parish.neil',
 'thomas.derek',
 'green.chris',
 'morris.james',
 'shelbrooke.alec',
 'williamson.gavin',
 'main.anne',
 'jones.marcus',
 'johnson.boris',
 'tolhurst.kelly',
 'davies.chris',
 'mackintosh.david',
 'evans.nigel',
 'mccartney.jason',
 'penrose.john',
 'dowden.oliver',
 'hopkins.kris',
 'aldous.peter',
 'simpson.keith',
 'jenrick.robert',
 'timpson.edward',
 'pawsey.mark',
 'murray.sheryll',
 'howarth.gerald',
 'miller.maria',
 'vara.shailesh',
 'boles.nick',
 'buckland.robert',
 'syms.robert',
 'holloway.adam',
 'quin.jeremy',
 'malthouse.kit',
 'throup.maggie',
 'mowat.david',
 'harrington.richard',
 'stephenson.andrew',
 'mitchell.andrew',
 'amess.david',
 'whately.helen',
 'phillips.stephen',
 'howell.john',


In [137]:
def count3(text, last_name, first_name):
    return text.count(first_name + ' ' + last_name) + text.count(first_name + last_name)

def count(text):
    c = 0
    for p in politicians:
        last_name, first_name = p.split('.')
        c += count3(text, last_name, first_name)
    return c

In [138]:
partition_df['transcript'] = partition_df['transcript'].apply(lambda x: x.lower())
partition_df

,partition_id,date,transcript,EU
0,521,2-apr-2016,well... you are not reconciled and you now run...,EU
1,521,2-apr-2016,in my six months in the finance ministry i put...,EU
2,521,2-apr-2016,"but overshadowing much of the summit, the pros...",EU
3,521,2-apr-2016,a holding centre for migrants on the holding c...,EU
4,521,2-apr-2016,greece is pushing ahead with plans to start de...,EU
...,...,...,...,...
567,1284,31-may-2016,also tonight: this city wouldn’t be what it is...,EU
568,1284,31-may-2016,leaving the european union would allow britain...,EU
569,1284,31-may-2016,"and there he was, telling his own side just wh...",EU
570,1284,31-may-2016,with just over three weeks to the eu referendu...,EU


In [139]:
partition_df['politician count'] = partition_df['transcript'].apply(lambda x: count(x))

In [140]:
partition_df = partition_df.loc[partition_df['politician count'] != 0]

In [141]:
partition_df

,partition_id,date,transcript,EU,politician count
7,522,2-apr-2016,let’s speak to our political correspondent car...,EU,2
20,525,4-apr-2016,"david cameron, enjoying the sun in lanzarote, ...",EU,1
21,525,4-apr-2016,he was there recently doing various trade deal...,EU,2
58,530,8-apr-2016,panama... people don’t talk about panama... pe...,EU,3
71,539,7-apr-2016,they will have more on david cameron being cri...,EU,2
...,...,...,...,...,...
555,1280,30-may-2016,from foe to friend - david cameron and the new...,EU,1
559,1281,30-may-2016,campaigning intensifies forthe eu referendum -...,EU,2
561,1281,30-may-2016,"the remain campaign launched a guarantee card,...",EU,5
565,1283,28-may-2016,"in it they said, voters were promised it they ...",EU,1


In [142]:
partition_df.to_csv('bbc_54_may_apr_partitions.csv')

In [143]:
partition_df['politician count'].sum()

376